<a href="https://colab.research.google.com/github/iamkaustubh1019/Deep_Learning/blob/master/Attempt_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
from collections import defaultdict
import itertools
from os.path import join

# dataset params
def get_train_config(json_path):
    with open(json_path) as json_file:
        config = json.load(json_file)
    return config

def get_train_configs(json_path):
    with open(json_path) as json_file:
        config = json.load(json_file)
        # find free hyper-params
        free_hp = []
        free_fields = []
        for field in config:
            if isinstance(config[field], dict): # free hyper-param
                free_fields.append(field)
                free_vs = []
                for v in config[field].values():
                    free_vs.append(v)
                free_hp.append(free_vs)

        if len(free_fields) == 0:
            return [config], ['.']

        # generate all combinations of hp
        free_hp_comb_list = list(itertools.product(*free_hp))

        # write the combination back to dict object
        ret_config_list = []
        ret_rel_path_list = []
        for free_hp_comb in free_hp_comb_list:
            ret_config = dict(config)
            rel_path = []
            # modify the free field with the value in each combination
            for field, v in zip(free_fields, free_hp_comb):
                ret_config[field] = v
                rel_path.append(field+'_'+str(v).replace('[', '').replace(']',''))
            ret_config_list.append(ret_config)
            ret_rel_path_list.append(join(*rel_path))
    return ret_config_list, ret_rel_path_list



Eval Files Execution shall start from here.


eval/cupy.utils.py

In [0]:
# Copyright (C) 2018  Mikel Artetxe <artetxem@gmail.com>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

import numpy

try:
    import cupy
except ImportError:
    cupy = None


def supports_cupy():
    return cupy is not None


def get_cupy():
    return cupy


def get_array_module(x):
    if cupy is not None:
        return cupy.get_array_module(x)
    else:
        return numpy


def asnumpy(x):
    if cupy is not None:
        return cupy.asnumpy(x)
    else:
        return numpy.asarray(x)


eval/embeddings.py

In [0]:
# Copyright (C) 2016-2018  Mikel Artetxe <artetxem@gmail.com>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.


import numpy as np

def read(file, threshold=0, vocabulary=None, dtype='float'):
    header = file.readline().strip().split(' ')
    count = int(header[0]) if threshold <= 0 else min(threshold, int(header[0]))
    dim = int(header[1])
    words = []
    matrix = np.empty((count, dim), dtype=dtype) if vocabulary is None else []
    for i in range(count):
        word, vec = file.readline().split(' ', 1)
        if vocabulary is None:
            words.append(word)
            matrix[i] = np.fromstring(vec, sep=' ', dtype=dtype)
        elif word in vocabulary:
            words.append(word)
            matrix.append(np.fromstring(vec, sep=' ', dtype=dtype))

    return (words, matrix) if vocabulary is None else (words, np.array(matrix, dtype=dtype))


def write(words, matrix, file):
    m = asnumpy(matrix)
    print('%d %d' % m.shape, file=file)
    for i in range(len(words)):
        print(words[i] + ' ' + ' '.join(['%.6g' % x for x in m[i]]), file=file)


def length_normalize(matrix):
    xp = get_array_module(matrix)
    norms = xp.sqrt(xp.sum(matrix**2, axis=1))
    norms[norms == 0] = 1
    return matrix / norms[:, xp.newaxis]


def mean_center(matrix):
    xp = get_array_module(matrix)
    avg = xp.mean(matrix, axis=0)
    return matrix - avg


def length_normalize_dimensionwise(matrix):
    xp = get_array_module(matrix)
    norms = xp.sqrt(xp.sum(matrix**2, axis=0))
    norms[norms == 0] = 1
    return matrix / norms


def mean_center_embeddingwise(matrix):
    xp = get_array_module(matrix)
    avg = xp.mean(matrix, axis=1)
    return matrix - avg[:, xp.newaxis]


eval_translation.py

In [13]:
# Copyright (C) 2016-2018  Mikel Artetxe <artetxem@gmail.com>
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

!pip install embeddings
import embeddings
import argparse
import collections
import numpy as np
import sys


BATCH_SIZE = 500


def main(src_embeddings,trg_embeddings):
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Evaluate embeddings of two languages in a shared space in word translation induction')
    parser.add_argument('src_embeddings', help='the source language embeddings')
    parser.add_argument('trg_embeddings', help='the target language embeddings')
    parser.add_argument('-d', '--dictionary', default=sys.stdin.fileno(), help='the test dictionary file (defaults to stdin)')
    parser.add_argument('--retrieval', default='nn', choices=['nn', 'invnn', 'invsoftmax'], help='the retrieval method (nn: standard nearest neighbor; invnn: inverted nearest neighbor; invsoftmax: inverted softmax)')
    parser.add_argument('--inv_temperature', default=1, type=float, help='the inverse temperature (only compatible with inverted softmax)')
    parser.add_argument('--inv_sample', default=None, type=int, help='use a random subset of the source vocabulary for the inverse computations (only compatible with inverted softmax)')
    parser.add_argument('--dot', action='store_true', help='use the dot product in the similarity computations instead of the cosine')
    parser.add_argument('--encoding', default='utf-8', help='the character encoding for input/output (defaults to utf-8)')
    parser.add_argument('--seed', type=int, default=0, help='the random seed')
    parser.add_argument('--precision', choices=['fp16', 'fp32', 'fp64'], default='fp64', help='the floating-point precision (defaults to fp64)')
    parser.add_argument('--cuda', action='store_true', help='use cuda (requires cupy)')
    args = parser.parse_args()

    # Choose the right dtype for the desired precision
    if args.precision == 'fp16':
        dtype = 'float16'
    elif args.precision == 'fp32':
        dtype = 'float32'
    elif args.precision == 'fp64':
        dtype = 'float64'

    # Read input embeddings
    srcfile = open(args.src_embeddings, encoding=args.encoding, errors='surrogateescape')
    trgfile = open(args.trg_embeddings, encoding=args.encoding, errors='surrogateescape')
    src_words, x = embeddings.read(srcfile, dtype=dtype)
    trg_words, z = embeddings.read(trgfile, dtype=dtype)

    # NumPy/CuPy management
    if args.cuda:
        if not supports_cupy():
            print('ERROR: Install CuPy for CUDA support', file=sys.stderr)
            sys.exit(-1)
        xp = get_cupy()
        x = xp.asarray(x)
        z = xp.asarray(z)
    else:
        xp = np
    xp.random.seed(args.seed)

    # Length normalize embeddings so their dot product effectively computes the cosine similarity
    if not args.dot:
        x = embeddings.length_normalize(x)
        z = embeddings.length_normalize(z)

    # Build word to index map
    src_word2ind = {word: i for i, word in enumerate(src_words)}
    trg_word2ind = {word: i for i, word in enumerate(trg_words)}

    # Read dictionary and compute coverage
    f = open(args.dictionary, encoding=args.encoding, errors='surrogateescape')
    src2trg = collections.defaultdict(set)
    oov = set()
    vocab = set()
    for line in f:
        src, trg = line.split()
        try:
            src_ind = src_word2ind[src]
            trg_ind = trg_word2ind[trg]
            src2trg[src_ind].add(trg_ind)
            vocab.add(src)
        except KeyError:
            oov.add(src)
    src = list(src2trg.keys())
    oov -= vocab  # If one of the translation options is in the vocabulary, then the entry is not an oov
    coverage = len(src2trg) / (len(src2trg) + len(oov))

    # Find translations
    translation = collections.defaultdict(int)
    if args.retrieval == 'nn':  # Standard nearest neighbor
        for i in range(0, len(src), BATCH_SIZE):
            j = min(i + BATCH_SIZE, len(src))
            similarities = x[src[i:j]].dot(z.T)
            nn = similarities.argmax(axis=1).tolist()
            for k in range(j-i):
                translation[src[i+k]] = nn[k]
    elif args.retrieval == 'invnn':  # Inverted nearest neighbor
        best_rank = np.full(len(src), x.shape[0], dtype=int)
        best_sim = np.full(len(src), -100, dtype=dtype)
        for i in range(0, z.shape[0], BATCH_SIZE):
            j = min(i + BATCH_SIZE, z.shape[0])
            similarities = z[i:j].dot(x.T)
            ind = (-similarities).argsort(axis=1)
            ranks = asnumpy(ind.argsort(axis=1)[:, src])
            sims = asnumpy(similarities[:, src])
            for k in range(i, j):
                for l in range(len(src)):
                    rank = ranks[k-i, l]
                    sim = sims[k-i, l]
                    if rank < best_rank[l] or (rank == best_rank[l] and sim > best_sim[l]):
                        best_rank[l] = rank
                        best_sim[l] = sim
                        translation[src[l]] = k
    elif args.retrieval == 'invsoftmax':  # Inverted softmax
        sample = xp.arange(x.shape[0]) if args.inv_sample is None else xp.random.randint(0, x.shape[0], args.inv_sample)
        partition = xp.zeros(z.shape[0])
        for i in range(0, len(sample), BATCH_SIZE):
            j = min(i + BATCH_SIZE, len(sample))
            partition += xp.exp(args.inv_temperature*z.dot(x[sample[i:j]].T)).sum(axis=1)
        for i in range(0, len(src), BATCH_SIZE):
            j = min(i + BATCH_SIZE, len(src))
            p = xp.exp(args.inv_temperature*x[src[i:j]].dot(z.T)) / partition
            nn = p.argmax(axis=1).tolist()
            for k in range(j-i):
                translation[src[i+k]] = nn[k]

    # Compute accuracy
    accuracy = np.mean([1 if translation[i] in src2trg[i] else 0 for i in src])
    print('Coverage:{0:7.2%}  Accuracy:{1:7.2%}'.format(coverage, accuracy))
    sys.exit("{0:.2f}".format(float(100)*accuracy))

if __name__ == '__main__':
    main()


  Stored in directory: /root/.cache/pip/wheels/17/7a/c7/29dae66bb181e6a9df35639e74c075c17b6627792077ab4696
Successfully built embeddings


usage: ipykernel_launcher.py [-h] [-d DICTIONARY]
                             [--retrieval {nn,invnn,invsoftmax}]
                             [--inv_temperature INV_TEMPERATURE]
                             [--inv_sample INV_SAMPLE] [--dot]
                             [--encoding ENCODING] [--seed SEED]
                             [--precision {fp16,fp32,fp64}] [--cuda]
                             src_embeddings trg_embeddings
ipykernel_launcher.py: error: the following arguments are required: trg_embeddings


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


UTILS


Call Backs


In [0]:
import math
from collections import defaultdict

class EarlyStopper():
    def __init__(self, patience, min_delta=1e-5):
        self.hist_loss = defaultdict(float)
        self.patience_cnt = 0
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')

    def check_early_stop(self, cur_loss, epoch):
        '''check if we should early stop'''
        self.hist_loss[epoch] = cur_loss
        # if epoch > 0 and self.hist_loss[epoch-1] - cur_loss > self.min_delta:
        if epoch > 0 and cur_loss < self.best_loss - self.min_delta:
            self.patience_cnt = 0
            self.best_loss = cur_loss
        else:
            self.patience_cnt += 1
        if self.patience_cnt > self.patience:
            print("early stopping...")
            return True
        else:
            return False

class ModelChecker():
    def __init__(self, min_delta=1e-5):
        self.best_loss = float('inf')
        self.epoch_loss = defaultdict(list)
        self.min_delta = min_delta

    def record_loss(self, cur_loss, epoch):
        self.epoch_loss[epoch].append(cur_loss)

    def get_best_loss(self):
        return self.best_loss

    def check_for_best(self, cur_loss, epoch):
        cur_loss = sum(self.epoch_loss[epoch]) / float(len(self.epoch_loss[epoch]))
        if epoch > 0 and cur_loss < self.best_loss - self.min_delta:
            self.best_loss = cur_loss
            return True
        else:
            return False

def SGDWR(T_total, T_0, T_mult, lr_max, lr_min): 
    # Use "STOCHASTIC GRADIENT DESCENT WITH WARM RESTARTS"
    # return a dictionary records at each epoch the learning rate: {<#epoch>:<lr>}
    lr_dict = {}
    T_cur = 0.0 # epochs since last restart
    T_i = float(T_0)
    for T in range(T_total+1):
        if T_cur > T_i:
            T_cur = 0.0
            T_i = T_i * T_mult
        lr = lr_min + 0.5*(lr_max-lr_min)*(1+math.cos(math.pi*(T_cur/T_i)))
        T_cur += 1
        lr_dict[T] = lr
    return lr_dict

In [25]:
!unsup-cross-lingual-embedding-transfer-master/data/download.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   278    0   278    0     0    776      0 --:--:-- --:--:-- --:--:--   776
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   278    0   278    0     0    920      0 --:--:-- --:--:-- --:--:--   917
bg
/dictionaries/en-bg.txt
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   297    0   297    0     0   4640      0 --:--:-- --:--:-- --:--:--  4640
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   297    0   297    0     0   4500      0 --:--:-- --:--:-- --:--:--  4500
/dictionaries/en-bg.0-500

Data Helper

In [28]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#

"""
USAGE:
OUTPUT:

"""

import torch
import os
import numpy as np
import random
from os import listdir 
from os.path import join, isdir
from tqdm import tqdm
tqdm.monitor_interval = 0
from numpy import linalg as LA
from collections import defaultdict
import errno
from sklearn import preprocessing
from scipy import stats
from itertools import cycle

def _handle_zeros_in_scale(scale, copy=True):
    ''' Makes sure that whenever scale is zero, we handle it correctly.
    This happens in most scalers when we have constant features.'''

    # if we are fitting on 1D arrays, scale might be a scalar
    if np.isscalar(scale):
        if scale == .0:
            scale = 1.
        return scale
    elif isinstance(scale, np.ndarray):
        if copy:
            # New array to avoid side-effects
            scale = scale.copy()
        scale[scale == 0.0] = 1.0
        return scale

def my_scale(X, axis=0):
    '''From scikit-learn preprocessing.scale'''
    Xr = np.asarray(X)
    mean_ = np.mean(X, axis)
    scale_ = np.std(X, axis)
    scale_ = _handle_zeros_in_scale(scale_, copy=False)

    Xr -= mean_
    Xr /= scale_

    return Xr, mean_, scale_

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def file_len(fname):
    with open(fname, errors='surrogateescape') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def load_bi_dict(fname, splitter='\t'):
    bi_dict = defaultdict(list)
    for l in open(fname, errors='surrogateescape'):
        if splitter in l:
            ss = l.strip().split(splitter)
        else:
            ss = l.strip().split(' ')
        bi_dict[ss[0]].append(ss[1])
    return bi_dict

def load_text_vec(fname, splitter=' ', vocab_size=None, top_n=None, norm=None):
    """
    Load dx1 word vecs from word2vec-like format:
    <word1> <dim1> <dim2> ...
    <word2> <dim1> <dim2> ...
    ...
    """
    # word_vecs = defaultdict(list)
    word_vecs = dict()
    words = []
    vecs = []
    with open(fname, "r", errors='surrogateescape') as f:
        if vocab_size is None:     
            vocab_size = file_len(fname)
        layer1_size = None
        vocab_count = 0

        for line in tqdm(f.readlines()[:min(vocab_size,top_n)+1]):
            ss = line.split(' ')
            if len(ss) <= 3:
                continue
            word = ss[0]
            dims = ' '.join(ss[1:]).strip().split(splitter)
            if layer1_size is None:
                layer1_size = len(dims)
                # print dims
                print("reading word2vec at vocab_size:%d, dimension:%d" % (vocab_size, layer1_size))

            vec = np.fromstring(' '.join(dims), dtype='float32', count=layer1_size, sep=' ')
            vecs.append(vec)
            words.append(word)
            vocab_count += 1
            if vocab_count >= vocab_size:
                break
            if top_n is not None:
                if vocab_count >= top_n:
                    break

        vecs = np.asarray(np.stack(vecs, axis=0))
        if norm == 'scale':
            vecs = preprocessing.scale(vecs)
        elif norm == 'l2':
            vecs = preprocessing.normalize(vecs, norm='l2', axis=1)
        elif norm == 'l2+mean_center':
            vecs = preprocessing.normalize(vecs, norm='l2', axis=1)
            vecs = mean_center(vecs)
        elif norm == 'none':
            pass
        else:
            print('ERROR: unrecoginized norm optioin {}'.format(norm))
        # print("vecs.mean(axis=0)", vecs.mean(axis=0))

        scaled_vecs, mean, std = my_scale(vecs)

        # vecs = np.copy(scaled_vecs)
        # vecs = preprocessing.normalize(vecs, norm='l2', axis=1)
        # vecs *= std
        # vecs += mean

        for i, word in enumerate(words):
            word_vecs[word] = vecs[i,:]

        print('vecs.shape:', vecs.shape)
    return vocab_size, word_vecs, words, vecs, scaled_vecs, mean, std, layer1_size

# def cycle_zip(A, B): # return zip of list A,B, which are in different sizes 
#     return zip(A, cycle(B)) if len(A) > len(B) else zip(cycle(A), B)

def downsample_frequent_words(counts, total_count, frequency_threshold=1e-3):
    if total_count > 1: # if inputs are counts
        threshold_count = float(frequency_threshold * total_count)
        probs = (np.sqrt(counts / threshold_count) + 1) * (threshold_count / counts)
        probs = np.maximum(probs, 1.0)    #Zm: Originally maximum, which upsamples rare words
        probs *= counts
        probs /= probs.sum()
    elif total_count <= 1: # inputs are frequency already
        probs = np.power(counts, 0.75)
        probs /= probs.sum()
    return probs

def load_freq(p, top_n=None, splitter=' '):
    """
    Load word frequence from word count
    <word1> <count1> ...
    <word2> <count2> ...
    ...
    return <word:word_count dictionary>, <freq_list>
    """
    if p is None:
        # assume uniform distribution
        return None, np.ones(top_n)/top_n 

        # assume Zipf's law
        # freq = 1/(np.arange(top_n)+1)
        # return None, downsample_frequent_words(freq/np.sum(freq), 0)

    else:
        w_count = defaultdict(list)
        count_list = []
        for l in open(p):
            ss = l.strip().split(splitter)
            w_count[ss[0]].append(float(ss[1]))
            count_list.append(float(ss[1]))
            if len(count_list) >= top_n:
                break
        counts = np.asarray(count_list, dtype=np.float32)
        total_count = counts.sum()
    return w_count, downsample_frequent_words(counts, total_count)

def uniform_batch_iter(xs, batch_size=32, num_epochs=1, shuffle=True, full_batch=True, verbose=False):
    """
    Generates a batch iterator for a dataset. Use uniform sample from data.
    xs: list of numpy array data, must has equal size on axis 0
    batch_size: size of batch
    num_epochs: number of epochs
    shuffle: True will shuffle xs with same random indices, False will not shuffle
    full_batch: True to make use every batch has the same batch_size, False the last batch of each epoch may contain samples less than batch_size
    verbose: True to print epoch and batch size info on the run 
    """

    data_sizes = [x.shape[0] for x in xs]
    assert data_sizes[1:] == data_sizes[:-1] # make sure they are all equal
    data_size = data_sizes[0]
    num_batches_per_epoch = int((data_size-0.1)/batch_size) + 1
    # make data be n times size of batch size
    res_size = batch_size - data_size % batch_size
    if res_size > 0 and full_batch:
        shuffle_indices = np.random.choice(np.arange(data_size), size=res_size, replace=False)
        xs = [np.concatenate((x, x[shuffle_indices]), axis=0) for x in xs]
        data_size = xs[0].shape[0]
    # print(num_epochs*num_batches_per_epoch)
    for epoch in range(num_epochs):
        if shuffle: # Shuffle the data at each epoch
            xs_shuffled = list()
            shuffle_indices = np.random.permutation(np.arange(data_size)) # same shuffle for all data
            for i,x in enumerate(xs):    
                xs_shuffled.append(x[shuffle_indices])
        else:
            xs_shuffled = xs
        if verbose:
            sys.stdout.write("\rIn epoch >> " + str(epoch + 1))
            sys.stdout.flush()
            # print("In epoch >> " + str(epoch + 1), end='', flush=True)
            # print("num batches per epoch is: " + str(num_batches_per_epoch), end='', flush=True)
        
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            xs_batch = [x[start_index:end_index] for x in xs_shuffled]
            batch = (xs_batch, start_index, end_index, epoch)
            yield batch

def freq_sample_batch_iter(xs, ps, batch_size=32, num_epochs=1, verbose=False):
    """
    Generates a batch iterator for a dataset. Use frequence sample from ps.
    """
    data_size = max([x.shape[0] for x in xs]) # find the max data size
    num_batches_per_epoch = int((data_size-0.1)/batch_size) + 1
    for epoch in range(num_epochs):
        if verbose:
            print("In epoch >> " + str(epoch + 1))
            print("num batches per epoch is: " + str(num_batches_per_epoch))
        for batch_num in range(num_batches_per_epoch):
            xs_batch = []
            for x, p in zip(xs, ps):
                idx = np.random.choice(np.arange(x.shape[0]), size=batch_size, replace=True, p=p)
                xs_batch.append(x[idx,:])
            yield (xs_batch, None, None)


# utility function
def mean_center(matrix, axis=0):
    # print(type(matrix))
    if type(matrix) is np.ndarray:
        avg = np.mean(matrix, axis=axis, keepdims=True)
    else:
        avg = torch.mean(matrix, axis=axis, keepdims=True)
    return matrix - avg

def get_wordvec(filename, top_n=None):
    arr = []
    words = []
    count = 0
    for num, line in enumerate(open(filename)):
        if num == 0 and len(line.split()) < 4:
            continue
        word, vect = line.rstrip().split(' ', 1)
        arr.append(vect.split())
        # assert len(vect.split()) == 300
        words.append(word)
        count += 1
        if top_n is not None and count >= top_n:
            break
    return np.vstack(arr).astype(float), words
    # return mean_center(np.vstack(arr).astype(float), axis=0), words

def save_emb(filename, X, words):
    """
    save vectors X(np.array) and words to file f
    """
    f = open(filename, 'w', errors='surrogateescape')
    num_words = X.shape[0]
    assert num_words == len(words), "saved words is not aligned with saved vectors"
    f.write('{} {}\n'.format(num_words, X.shape[1]))
    for i,word in enumerate(words):
        word_vec_str = ' '.join(map(str, X[i,:].tolist()))
        f.write("{} {}\n".format(word, word_vec_str))

def get_dictionary_index(filename, src_words, tgt_words, limit=None, unique=False):
    src_idx, tgt_idx = [], []
    counter = 0
    for i,l in enumerate(open(filename).readlines()):
        if limit is not None and counter >= limit:
            break
        ss = l.strip().split()
        if ss[0].lower() in src_words and ss[1].lower() in tgt_words:
            if unique and (src_words.index(ss[0].lower()) in src_idx or tgt_words.index(ss[1].lower()) in tgt_idx):
                continue
            src_idx.append(src_words.index(ss[0].lower()))
            tgt_idx.append(tgt_words.index(ss[1].lower()))
            counter += 1
    return src_idx, tgt_idx

def get_dictionary_matrix(filename, src_words, tgt_words, limit=None):
    """
    Suppose the dictionary has line format: <src_word> <tgt_word>
    """
    F = np.zeros((len(src_words),len(tgt_words)))
    src_idx, tgt_idx = get_dictionary_index(filename, src_words, tgt_words, limit=limit)
    F[np.asarray(src_idx), np.asarray(tgt_idx)] = 1
    return F

def get_data(src, tgt, top_n=200000):
    src_arr, src_words = get_wordvec('./wiki.en.vec', top_n)
    tgt_arr, tgt_words = get_wordvec('./wiki.bg.vec', top_n)
    return src_arr, src_words, tgt_arr, tgt_words

def sparsify_mat(K, nn):
    ret_K = np.zeros(K.shape)
    for i in range(K.shape[0]):
        index = np.argsort(K[i, :])[-nn:]
        ret_K[i, index] = K[i, index]
    return ret_K

def plot_coo_matrix(m):
    if not isinstance(m, coo_matrix):
        m = coo_matrix(m)
    fig = plt.figure()
    ax = fig.add_subplot(111, axisbg='black')
    ax.plot(m.col, m.row, 's', color='white', ms=1)
    ax.set_xlim(0, m.shape[1])
    ax.set_ylim(0, m.shape[0])
    ax.set_aspect('equal')
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    return ax

def sym_sparsify_mat(K, nn):
        K_sp = sparsify_mat(K, nn)
        K_sp = (K_sp + K_sp.T) / 2  # in case of non-positive semi-definite
        return K_sp

def get_adj(basedir, src, tgt, nn, logger, normalize=True):
    logger.info('Loading data...')
    src_arr, tgt_arr = get_data(basedir, src, tgt)
    logger.info('Loading data finished')
    if normalize:
        src_arr = src_arr / np.linalg.norm(src_arr, ord=2, axis=1, keepdims=True)
        tgt_arr = tgt_arr / np.linalg.norm(tgt_arr, ord=2, axis=1, keepdims=True)
    src_adj = sym_sparsify_mat(src_arr.dot(src_arr.T), nn)
    tgt_adj = sym_sparsify_mat(tgt_arr.dot(tgt_arr.T), nn)
    logger.info('Sparsification finished')
    # print(type(src_adj), type(tgt_adj))
    return torch.from_numpy(src_adj.astype(float)), torch.from_numpy(tgt_adj.astype(float))

if __name__ == '__main__':
    pass
    basedir = "./unsup-cross-lingual-embedding-transfer-master/data"
    src = 'es'
    tgt = 'en'
    nn = 5

    src_arr, tgt_arr = get_data(basedir, src, tgt)

    src_adj = sym_sparsify_mat(src_arr.dot(src_arr.T), nn)
    tgt_adj = sym_sparsify_mat(tgt_arr.dot(tgt_arr.T), nn)
    print(src_adj.shape)
    print(tgt_adj.shape)
    # print src_adj[0]


TypeError: ignored

Model

Common_layers

In [0]:
import tensorflow as tf

USE_BN = True

def highway_layer(input_layer, input_dim, hidden_dim, output_dim, Wg=None, bg=None, Ws=None, bs=None, name='highway'):
    with tf.name_scope(name + "-gate-layer"):
        if Wg is None:
            Wg = tf.Variable(tf.truncated_normal([input_dim, input_dim], stddev=0.05), name="W")
        if bg is None:
            bg = tf.Variable(tf.truncated_normal([input_dim], stddev=0.05), name="b")
        g = tf.nn.sigmoid(tf.add(tf.matmul(input_layer, Wg), bg))

    trans, Ws, bs = dense_layers(input_layer, input_dim, hidden_dim, output_dim, Ws=Ws, bs=bs, name=name+'-dense-layer')
    out = tf.multiply(g, trans) + tf.multiply(1-g, input_layer)
    return out, Wg, bg, Ws, bs

def dense_layers(input_layer, input_dim, hidden_dim, output_dim, Ws=None, bs=None, name='fc', activation='elu', bias=True, orth_init=False, add_multiple_noise=None, BN=USE_BN, BN_phase=True, BN_reuse=None):
    if Ws is None:
        Ws = (len(hidden_dim)+1)*[None]
    if bs is None:
        bs = (len(hidden_dim)+1)*[None]
    dims = [input_dim] + hidden_dim + [output_dim]
    layers = [input_layer] + len(hidden_dim)*[None] + [None]
    with tf.name_scope(name):
        for i, d_i in enumerate(dims[:-1]):
            # add multiple noise to input layers
            if add_multiple_noise is not None:
                # print("add_multiple_noise", add_multiple_noise)
                gaussian_noise = tf.random_normal(shape = tf.shape(layers[i]), mean = 1.0, stddev = add_multiple_noise, dtype = tf.float32)
                layers[i] = tf.multiply(layers[i], gaussian_noise)
            # Wx + b
            d_o = dims[i+1]
            if Ws[i] is None:
                if orth_init:
                    initializer = tf.orthogonal_initializer
                else:
                    initializer = tf.contrib.layers.xavier_initializer()
                Ws[i] = tf.get_variable(name="W"+str(i)+name, shape=[d_i, d_o], dtype=tf.float32, initializer=initializer)
            if bs[i] is None and bias:
                bs[i] = tf.Variable(tf.constant(0.1, shape=[d_o], dtype=tf.float32), name="b"+str(i))
            elif not bias:
                bs[i] = tf.constant(0, shape=[d_o], dtype=tf.float32)

            layers[i+1] = tf.nn.xw_plus_b(layers[i], Ws[i], bs[i], name="fc_out"+str(i+1))

            if BN and i!=len(hidden_dim): # add BN
            # if BN:
                layers[i+1] = tf.contrib.layers.batch_norm(layers[i+1], center=True, scale=True, is_training=BN_phase, scope=name+'_bn_'+str(i), reuse=BN_reuse)

            if i!=len(hidden_dim) and activation:
                # layers[i+1] = tf.nn.tanh(layers[i+1]) # tanh activation for any other output
                if activation == 'elu':
                    layers[i+1] = tf.nn.elu(layers[i+1]) # elu activation for any other output 
                elif activation == 'relu':
                    layers[i+1] = tf.nn.relu(layers[i+1]) # relu activation for any other output
                elif activation == 'tanh':
                    layers[i+1] = tf.nn.tanh(layers[i+1]) # tanh activation for any other output

    return layers[len(hidden_dim)+1], Ws, bs 

def add_multiple_gaussian_noise(inp_tensor, stddev):
    gaussian_noise = tf.random_normal(shape = tf.shape(inp_tensor), mean = 1.0, stddev = stddev, dtype = tf.float32)
    return tf.multiply(inp_tensor, gaussian_noise)

Cycle_Align_Emb

In [16]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import os, sys
import numpy as np
#sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
from tqdm import tqdm
import errno

import tensorflow as tf
import tensorflow.contrib.layers as ly
from tensorflow.python import debug as tf_debug

from utils.callbacks import SGDWR, EarlyStopper, ModelChecker
from utils.eval_helper import knn_accuracy_from_list, find_common_words
from utils.visual_emb import plot_align_matrix, plot_emb
from utils.data_helper import load_text_vec, load_freq, uniform_batch_iter, freq_sample_batch_iter, sparsify_mat, mkdir_p

from model.distance import *
from model.common_layers import *
from model.cycle_emb import CycleEmb, USE_EUC_DIST, TIED_WEIGHTS
from model.sinkhorn import get_sinkhorn_distance
from model.wasserstein_gan import set_W_gan_layers

from numpy import linalg as LA
from functools import partial
import time
import warnings

USE_EUC_DIST_MATRIX = False
TRANS_ITER = 1
DISC_ITERS = 5 # for WGAN, we iterate discriminator
WGAN_TRANS_ITERS = 5 # for WGAN, we iterate transformation
DEBUG = False
LAMBDA_SH = 10
SINKHORN_LAYER_DEPTH = 20
MULTIPLE_NOISE_STD = None # add noise to sinkhorn layers
PATIENCE = 4000 # number of epochs we wait before call early stop
MIN_DELTA = 1e-5 # improvement larger than delta is considered as improvement

################################################################################
### The cross-lingual embeding model learned from cycle adversarial network
class CycleAlignEmb(CycleEmb):
    def __init__(self, trans_hidden_dim=[50], disc_hidden_dim=[50], recon_loss_weight=[1,1], constraint_loss_weight=1, init_align_loss_weight=1, trans_activation=False, lambda_sh=10, sinkhorn_layer_depth=20, cyc_loss=True, use_sinkhorn=True, use_BN=True, norm='scale', save_path=None, F_init=None):
        CycleEmb.__init__(self, trans_hidden_dim, disc_hidden_dim, recon_loss_weight, trans_activation, cyc_loss, use_sinkhorn, use_BN, norm, save_path)
        self.constraint_loss_weight = constraint_loss_weight # relative weight of loss of F constraint
        self.init_align_loss_weight = init_align_loss_weight # if we have initial seed, the weight of that part of loss
        self.lambda_sh = lambda_sh
        self.sinkhorn_layer_depth = sinkhorn_layer_depth
        self.F_init = F_init
        self.cyc_loss = cyc_loss

    def build_WGAN_layer(self, ):
        if self.F_init is None:
            l_disc_s, l_gen_s, self.Ws_s_vs_fs, self.bs_s_vs_fs = set_W_gan_layers(self.input_s, self.trans_t, input_dim=self.emb_dim_list[0], disc_hidden_dim=self.disc_hidden_dim, scale=10, name='W_gan_s')
            l_disc_t, l_gen_t, self.Ws_t_vs_ft, self.bs_t_vs_ft = set_W_gan_layers(self.input_t, self.trans_s, input_dim=self.emb_dim_list[1], disc_hidden_dim=self.disc_hidden_dim, scale=10, name='W_gan_t')
            if self.cyc_loss:
                self.l_disc = (l_disc_s + l_disc_t)/2
                self.l_gen = (l_gen_s + l_gen_t)/2
            else:
                self.l_disc = l_disc_s
                self.l_gen = l_gen_s
        else:
            self.l_disc = tf.constant(0, dtype=tf.float32)
            self.l_gen = tf.constant(0, dtype=tf.float32)

    def build_sinkhorn_layer(self, lambda_sh=LAMBDA_SH, sinkhorn_layer_depth=SINKHORN_LAYER_DEPTH, multiple_noise_std=None):
        '''
        Following "Marco Cuturi, Sinkhorn Distances: Lightspeed Computation of Optimal Transport, NIPS 2013"
        '''
        if multiple_noise_std is not None:
            self.input_t_inp = add_multiple_gaussian_noise(self.input_t, multiple_noise_std)
            self.trans_s_inp = add_multiple_gaussian_noise(self.trans_s, multiple_noise_std)
            self.trans_t_inp = add_multiple_gaussian_noise(self.trans_t, multiple_noise_std)
            self.input_s_inp = add_multiple_gaussian_noise(self.input_s, multiple_noise_std)
        else:
            self.input_t_inp = self.input_t
            self.trans_s_inp = self.trans_s
            self.trans_t_inp = self.trans_t
            self.input_s_inp = self.input_s

        M1 = euclidean_distance_matrix(self.trans_s_inp, self.input_t_inp) if USE_EUC_DIST_MATRIX else norm_euclidean_distance_matrix(self.trans_s_inp, self.input_t_inp)
        M2 = euclidean_distance_matrix(self.input_s_inp, self.trans_t_inp) if USE_EUC_DIST_MATRIX else norm_euclidean_distance_matrix(self.input_s_inp, self.trans_t_inp)
        
        if self.cyc_loss:
            self.sinkhorn_distance = (get_sinkhorn_distance(M1, self.freq_s, self.freq_t, lambda_sh=lambda_sh, depth=sinkhorn_layer_depth) + \
                                  get_sinkhorn_distance(M2, self.freq_s, self.freq_t, lambda_sh=lambda_sh, depth=sinkhorn_layer_depth))/2
        else:
            self.sinkhorn_distance = get_sinkhorn_distance(M2, self.freq_s, self.freq_t, lambda_sh=lambda_sh, depth=sinkhorn_layer_depth)

    def build_F_init_layer(self):
        if self.F_init is not None:
            t_nz_idx, s_nz_idx = np.where(self.F_init>0)
            t_nz_idx, s_nz_idx = np.sort(t_nz_idx), np.sort(s_nz_idx)
            self.F_init = self.F_init[t_nz_idx,:]
            self.F_init = self.F_init[:,s_nz_idx]
            self.F_init_row=self.F_init/np.sum(self.F_init, axis=1, keepdims=True)
            self.F_init_col=np.transpose(self.F_init/np.sum(self.F_init, axis=0, keepdims=True))
            # print('self.F_init', self.F_init)
            # print('self.F_init.shape', self.F_init.shape)
            # take care of the initial dictionary
            # F_init_count = np.count_nonzero(self.F_init)
            # print('F_init_count', F_init_count)
            F_init_row = tf.convert_to_tensor(self.F_init_row, dtype=tf.float32)
            F_init_col = tf.convert_to_tensor(self.F_init_col, dtype=tf.float32)

            self.init_vecs_list = [None, None]
            self.init_vecs_list[0] = self.vecs_list[0][s_nz_idx,:]
            self.init_vecs_list[1] = self.vecs_list[1][t_nz_idx,:]

            if USE_EUC_DIST:
                self.align_loss_row_init = euclidean_distance(self.trans_s_init, self.input_t_init) / tf.cast(self.F_init.shape[0], dtype=tf.float32)
                self.align_loss_col_init = euclidean_distance(self.trans_t_init, self.input_s_init) / tf.cast(self.F_init.shape[1], dtype=tf.float32)
            else:
                self.align_loss_row_init = 1 - cosine_similarity(tf.matmul(F_init_row, self.trans_s_init), self.input_t_init) / tf.cast(self.F_init.shape[0], dtype=tf.float32)
                self.align_loss_col_init = 1 - cosine_similarity(tf.matmul(F_init_col, self.trans_t_init), self.input_s_init) / tf.cast(self.F_init.shape[1], dtype=tf.float32)
            self.init_align_loss = (self.align_loss_row_init + self.align_loss_col_init)/2
        else:
            self.init_align_loss = tf.constant(0, dtype=tf.float32)

    def build_model(self):
        """ 
        build the graph for training
        internal dimentions:
        bs: batch_size, ed: embedding dimension, hd: hidden dimension
        """
        self.build_input_layers()
        self.build_transfer_layers()

        # alignment loss
        self.build_sinkhorn_layer(lambda_sh=self.lambda_sh, sinkhorn_layer_depth=self.sinkhorn_layer_depth, multiple_noise_std=MULTIPLE_NOISE_STD)
        self.build_F_init_layer()
        self.build_WGAN_layer()

        # recontruction loss
        self.build_recontruction_layers()
        self.l_trans_wgan = self.l_gen + self.recon_loss_weight[0]*self.reconstruct_loss
        self.l_trans_sh = self.sinkhorn_distance + self.recon_loss_weight[1]*self.reconstruct_loss + self.init_align_loss_weight*self.init_align_loss

        self.set_summary()

    def set_summary(self):
        tf.summary.scalar('discriminator loss', self.l_disc)
        tf.summary.scalar('wgan transformation(generator) loss', self.l_trans_wgan)
        tf.summary.scalar('sinkhorn transformation loss', self.l_trans_sh)
        tf.summary.scalar('reconstruct loss', self.reconstruct_loss)
        tf.summary.scalar('init alignment loss', self.init_align_loss)
        tf.summary.scalar('sinkhorn distance', self.sinkhorn_distance)
        if len(self.trans_hidden_dim) == 0:
            W = self.Ws_s2t[0]
            self.orth_loss = tf.norm(tf.matmul(W, W, transpose_a=True)-tf.eye(self.emb_dim_list[0]), ord='fro', axis=(0,1))
            tf.summary.scalar('orth_loss', self.orth_loss)
        self.merged_summary = tf.summary.merge_all()

    def train(self, batch_size=128, lr=0.001, epoch=10000, wgan_epoch=2000, logger=None, validation=None, dp_keep_prob=1.0, patience=PATIENCE, min_delta = MIN_DELTA):
        """ 
        train with adversarial/sinkhorn loss
        log: log file to record training
        validation: [tgt_words, true_src_words]
        """
        batch_per_epoch = int(self.vecs_list[0].shape[0]/batch_size) + 1 # how many steps in a epoch
        total_steps = batch_per_epoch * epoch
        early_stopper = EarlyStopper(patience*batch_per_epoch, min_delta)
        model_checker = ModelChecker()

        learning_rate = tf.placeholder(tf.float32, shape=[])
        lr_schedule_dict = SGDWR(T_total=epoch, T_0=10, T_mult=1.2, lr_max=lr, lr_min=lr/100)

        if self.F_init is None: # we will switch from WGAN to Sinkhorn during training
            swith_flag = False # first WGAN and then Sinkhorn
            # swith_flag = True # Use Sinkhorn all the time

        with tf.Session(config=self.session_conf) as sess:
            # sess = tf_debug.LocalCLIDebugWrapperSession(sess)
            # sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
            self.writer = tf.summary.FileWriter(os.path.join(self.save_path, 'graph'), graph=sess.graph)
            if self.saver is None:
                self.saver = tf.train.Saver(tf.global_variables())
                # self.saver = tf.train.Saver(trans_variables)
            global_step = tf.Variable(0, name="global_step", trainable=False)
            
            # define train operation
            if TIED_WEIGHTS:
                trans_variables = self.Ws_s2t
            else:
                trans_variables = self.Ws_s2t + self.Ws_t2s
            if self.F_init is None:
                disc_variables = self.Ws_s_vs_fs + self.bs_s_vs_fs + self.Ws_t_vs_ft + self.bs_t_vs_ft

            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                sh_trans_optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
                # cliping the weight norm
                # grads_and_vars = sh_trans_optimizer.compute_gradients(self.l_trans_sh)
                # capped_grads_and_vars = [(tf.clip_by_norm(gv[0], clip_norm=10, axes=0), gv[1])
                #              for gv in grads_and_vars]
                # sh_trans_optimizer = sh_trans_optimizer.apply_gradients(capped_grads_and_vars)
                sh_trans_train_op = sh_trans_optimizer.minimize(self.l_trans_sh, global_step=global_step, var_list=trans_variables)

                if self.F_init is None:
                    wgan_trans_optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate*10)
                    wgan_trans_train_op = wgan_trans_optimizer.minimize(self.l_trans_wgan, global_step=global_step, var_list=trans_variables)
                    disc_optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
                    disc_train_op = disc_optimizer.minimize(self.l_disc, var_list=disc_variables)

            sess.run(tf.global_variables_initializer())
            step = 0

            batches_src = uniform_batch_iter([self.vecs_list[0], self.freq_list[0]], batch_size=batch_size, num_epochs=epoch, shuffle=True, full_batch=True, verbose=False)
            batches_tgt = uniform_batch_iter([self.vecs_list[1], self.freq_list[1]], batch_size=batch_size, num_epochs=epoch, shuffle=True, full_batch=True, verbose=False)
            val_acc = 0.0
            cur_lr = lr
            epoch_start_time = time.time()

            for batch_src, batch_tgt in zip(batches_src, batches_tgt):
                x_src_batch = batch_src[0][0]
                x_tgt_batch = batch_tgt[0][0]
                freq_src_batch = batch_src[0][1]
                freq_tgt_batch = batch_tgt[0][1]

                # more advenced lr scheduler
                # T = int(step/batch_per_epoch) + 1
                # cur_lr = lr_schedule_dict[T]    
                '''OR'''
                # if step < total_steps/2:
                #     cur_lr = lr
                # else:
                #     cur_lr = lr/10
                '''OR'''
                # simplest lr scheduler
                # cur_lr = lr

                feed_dict = {
                  self.input_s: x_src_batch,
                  self.input_t: x_tgt_batch,
                  self.dp_keep_prob: dp_keep_prob,
                  self.freq_s: freq_src_batch,
                  self.freq_t: freq_tgt_batch,
                  learning_rate: cur_lr,
                  self.phase: True,
                }
                if self.F_init is not None:
                    feed_dict[self.input_s_init] = self.init_vecs_list[0]
                    feed_dict[self.input_t_init] = self.init_vecs_list[1]

                if self.F_init is None and (not swith_flag): 
                    
                    # update WGAN disc models
                    for _ in range(DISC_ITERS):
                        _, l_disc, l_trans_wgan = sess.run([disc_train_op, self.l_disc, self.l_trans_wgan], feed_dict)
                    # print("l_trans_wgan before wgan_trans_train_op", l_trans_wgan)

                    # update WGAN transfer models
                    # for _ in range(WGAN_TRANS_ITERS):
                    _, step, loss_reconstruct, loss_comb, l_trans_wgan, summary = sess.run([wgan_trans_train_op, global_step, self.reconstruct_loss, self.sinkhorn_distance, self.l_trans_wgan, self.merged_summary], feed_dict)
                    # print("l_trans_wgan after wgan_trans_train_op", l_trans_wgan)

                    # judge convergence, if so, switch training loss
                    # if early_stopper.check_early_stop(loss_comb, int( (step-0.1) / batch_per_epoch)+1) or step > total_steps/2:
                    if step > wgan_epoch * batch_per_epoch:
                        swith_flag = True
                        if not self.use_sinkhorn:
                            logger.info('Epoch, {0}, end up just using WGAN!'.format(int( (step-0.1) / batch_per_epoch)+1))
                            break
                        logger.info('Epoch, {0}, switch to sinkhorn loss!'.format(int( (step-0.1) / batch_per_epoch)+1))
                        self.saver.restore(sess, os.path.join(self.save_path, 'model.ckpt'))
                        early_stopper = EarlyStopper(patience*batch_per_epoch, min_delta)
                        model_checker = ModelChecker()
                        cur_lr = lr
                elif self.use_sinkhorn: # update sinkhorn transfer models
                    for _ in range(TRANS_ITER):
                        _, step, loss_reconstruct, loss_comb, init_align_loss, summary = sess.run([sh_trans_train_op, global_step, self.reconstruct_loss, self.l_trans_sh, self.init_align_loss, self.merged_summary], feed_dict)

                    # handle early stopping
                    # if step % batch_per_epoch == 0:
                    if early_stopper.check_early_stop(loss_comb, int( (step-0.1) / batch_per_epoch)+1):
                        logger.info('Epoch, {0} early stopping!'.format(int( (step-0.1) / batch_per_epoch)+1))
                        print('Epoch, {0} early stopping!'.format(int( (step-0.1) / batch_per_epoch)+1))
                        break
                else:
                    print('ERROR: Invalid config for training!')

                # validation and print
                if step % (50*batch_per_epoch) == 0:
                    validation_start_time = time.time()
                    stdout_buffer = ''
                    self.writer.add_summary(summary, step)
                    stdout_buffer += 'VALIDATION: Epoch:{} '.format(int( (step-0.1) / batch_per_epoch)+1)

                    if DEBUG:
                        for w in ['城市', '小行星', '文学']:
                            w_knn = self.find_k_nearest(w=w, k=5, src2tgt=True)
                            print('word: %s has knn words --- %s' % (w, ' '.join(w_knn)))
                    if validation is not None:
                        self.transfer_tgt_emb(sess=sess)
                        # print('validation[0][:10]:', validation[0][:10])
                        # print('validation[1][:10]:', validation[1][:10])
                        nnr_tgt = self.find_nearest_gpu(validation[0], src2tgt=True, batch_size=batch_size, sess=sess)
                        # print('nnr_tgt[:10]:', nnr_tgt[:10])
                        val_acc = knn_accuracy_from_list(nnr_tgt, validation[1], k=1)
                        stdout_buffer += 'Bilex acc:{0:.4f} '.format(val_acc)
                        val_acc_value = tf.Summary.Value(tag='val_acc', simple_value=val_acc)
                        self.writer.add_summary(tf.Summary(value=[val_acc_value]), step)

                    stdout_buffer += 'Validation eclipsed time \n'.format(time.time()-validation_start_time)
                    sys.stdout.write(stdout_buffer)
                    sys.stdout.flush()

                # check objective to save the best model
                model_checker.record_loss(loss_comb, int( (step-0.1) / batch_per_epoch)+1)
                if step % batch_per_epoch == 0:
                    print('Epoch  {}, time eclipsed {}'.format(int((step-0.1) / batch_per_epoch)+1, time.time()-epoch_start_time))
                    epoch_start_time = time.time()
                    if model_checker.check_for_best(loss_comb, int((step-0.1) / batch_per_epoch)+1):
                        save_path = self.saver.save(sess, os.path.join(self.save_path, 'model.ckpt'))
                        best_loss = model_checker.get_best_loss()
                        print('Saved at combined loss {}'.format(best_loss))
                        if validation is not None and logger is not None:
                            logger.info('Saved at epoch:{0}, step:{1} loss:{2:.4f} Bilingual Induction Accuracy --- {3:.4f}'.format(int( (step-0.1) / batch_per_epoch)+1, step, best_loss, val_acc))
                        cur_lr = lr
                    else:
                        cur_lr = lr*0.95

        self.writer.close()
        return val_acc

ModuleNotFoundError: ignored